In [ ]:
# Define the function to check Maxwell's count for all subgraphs with more than one edge
def check_maxwells_count_for_subgraphs(graph):
    vertices = graph.vertices()  # Get all vertices of the graph
    
    # Iterate over all subsets of vertices of size 2 or more
    for size in range(2, len(vertices) + 1):
        for subgraph_vertices in Subsets(vertices, size):
            subgraph = graph.subgraph(subgraph_vertices)  # Get subgraph induced by vertex subset
            
            # Only consider subgraphs with more than 1 edge
            if len(subgraph.edges()) > 1:
                E = len(subgraph.edges())  # Number of edges in the subgraph
                V = len(subgraph.vertices())  # Number of vertices in the subgraph
                
                # Check if Maxwell's count: E = 3V - 6
                if E > 3 * V - 6:
                    return False, subgraph  
    return True, None  # If all subgraphs satisfy the condition
# Edge list of double-banana graph
edges = [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6),
         (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6),
        (1, 2), (2, 3), (1, 3), (4, 5), (5, 6), (6, 4)]

# Create the graph
G = Graph(edges)

# Check if all subgraphs satisfy Maxwell's count
result, violating_subgraph = check_maxwells_count_for_subgraphs(G)

if result:
    print("All subgraphs with more than 1 edge satisfy Maxwell's count.")
else:
    print("A subgraph does not satisfy Maxwell's count. Subgraph details:")
    print("Subgraph vertices:", violating_subgraph.vertices())
    print("Subgraph edges:", violating_subgraph.edges())


All subgraphs with more than 1 edge satisfy Maxwell's count.


In [ ]:
from itertools import combinations

# Generator function to generate all induced subgraphs of a graph
def generate_induced_subgraphs(graph):
    vertices = graph.vertices()
    for size in range(5, len(vertices) + 1): # The smallest banana-shape graph need 5 vertices
        for vertex_subset in combinations(vertices, size):
            yield graph.subgraph(vertex_subset)  # Use a generator for efficiency

# Function to check if a subgraph satisfies Maxwell's count (E = 3V - 6)
def check_maxwells_count(E, V):
    return E == 3 * V - 6

# Function to check if the union of subgraphs satisfies Maxwell's count
def check_union_maxwells_count(subgraphs):
    all_edges = set()
    all_vertices = set()
    
    for subgraph in subgraphs:
        all_edges.update(subgraph.edges())
        all_vertices.update(subgraph.vertices())
    
    E = len(all_edges)
    V = len(all_vertices)
    return E == 3 * V - 6

# Function to check if a combination of subgraphs is connected
def check_connectivity(subgraphs):
    all_edges = set()
    all_vertices = set()

    for subgraph in subgraphs:
        all_edges.update(subgraph.edges())
        all_vertices.update(subgraph.vertices())

    union_graph = Graph()
    union_graph.add_edges(all_edges)
    return union_graph.is_connected()

# Function to check the shared vertices condition with exact match
def check_shared_vertices(subgraphs):
    # Collect all vertices from each subgraph
    vertex_sets = [set(subgraph.vertices()) for subgraph in subgraphs]
    
    # Find the union of all vertices
    all_vertices = set.union(*vertex_sets)
    
    # Count shared vertices (appear in more than one subgraph)
    shared_vertex_count = sum(
        sum(vertex in vertex_set for vertex_set in vertex_sets) > 1
        for vertex in all_vertices
    )
    
    # Check if the number of shared vertices equals the size of the combination
    return shared_vertex_count == len(subgraphs)

# Main function to check if the graph is rigid
def check_rigid_graph(graph):
    # Generate all subgraphs of the graph using a generator
    all_subgraphs = generate_induced_subgraphs(graph)

    # Filter subgraphs that satisfy Maxwell's count
    valid_subgraphs = [sg for sg in all_subgraphs if check_maxwells_count(len(sg.edges()), len(sg.vertices()))]

    # If there are fewer than 2 valid subgraphs, the graph is rigid
    if len(valid_subgraphs) < 2:
        return True, []  # Rigid, no possible combinations

    # Now check combinations of valid subgraphs
    valid_combos = []

    for size in range(2, 3):  # Combinations of size 2 or more, here I limit the maximum size to 3 to make the algorithm less computationally expensive
        for combo in combinations(valid_subgraphs, size):
            if check_union_maxwells_count(combo):  # Check Maxwell's count for the union
                if check_connectivity(combo):  # Check if the combination is connected
                    if check_shared_vertices(combo):  # Check shared vertices condition
                        valid_combos.append(combo)

    # If valid combinations exist, the graph is flexible
    if valid_combos:
        return False, valid_combos  # Flexible, output valid combinations

    # Otherwise, the graph is rigid
    return True, []  # Rigid, no valid combinations

# Edge list of double-banana graph
edges = [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6),
         (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6),
        (1, 2), (2, 3), (1, 3), (4, 5), (5, 6), (6, 4)]

# Create the graph
G = Graph(edges)

# Check if the graph is rigid
is_rigid, valid_combos = check_rigid_graph(G)

if is_rigid:
    print("The graph is rigid.")
else:
    print("The graph is not rigid. Valid combinations of subgraphs:")
    for combo in valid_combos:
        print("Combination:")
        for subgraph in combo:
            print(f"Subgraph vertices: {subgraph.vertices()}, edges: {subgraph.edges()}")


The graph is not rigid. Valid combinations of subgraphs:
Combination:
Subgraph vertices: [0, 1, 2, 3, 7], edges: [(0, 1, None), (0, 2, None), (0, 3, None), (1, 2, None), (1, 3, None), (1, 7, None), (2, 3, None), (2, 7, None), (3, 7, None)]
Subgraph vertices: [0, 4, 5, 6, 7], edges: [(0, 4, None), (0, 5, None), (0, 6, None), (4, 5, None), (4, 6, None), (4, 7, None), (5, 6, None), (5, 7, None), (6, 7, None)]


In [14]:

def Conjecture(graph):
    # Check Maxwell's count condition for subgraphs with more than one edge
    result, _ = check_maxwells_count_for_subgraphs(graph)
    if not result:
        return 0  # If any subgraph violates Maxwell's count, return non-rigid (0)

    # Check the rigidity condition
    is_rigid, _ = check_rigid_graph(graph)
    return 1 if is_rigid else 0  # Return 1 if rigid, 0 if not rigid



In [ ]:
#NEXT, LETS RUN OUR CONJECTURE ON ALL CIRITICAL GRAPHS OF 8 VERTICES AND COMPARE THE RESULT WITH WHAT WE GET FROM USING THE RIGIDITY MATRIX

In [11]:
import numpy as np
from sage.all import Matrix, QQ, vector  # Import from SageMath
from sage.graphs.graph_generators import graphs

# Define initial 3D vertex coordinates
vertices = [
    vector([30/10, -40/17, -5/36]),  # p1
    vector([0, -1, 0]),              # p2
    vector([1/3, 15/46, -3/4]),      # p3
    vector([8/17, -9/17, -1/6]),     # p4
    vector([0, 3, 0]),               # p5
    vector([-18/10, -32/15, 34/19]), # p6
    vector([-21/17, -31/13, 32/19]), # p7
    vector([11/23, 17/49, -31/42])  # p8
]

# Function to build the rigidity matrix for a set of vertices and edges
def build_rigidity_matrix(vertices, edges):
    R = Matrix(QQ, len(edges), len(vertices) * 3)  # Create the matrix based on edges and vertex dimensions
    for k, (i, j) in enumerate(edges):
        for dim in range(3):
            R[k, 3*i + dim] = vertices[j][dim] - vertices[i][dim]
            R[k, 3*j + dim] = -vertices[j][dim] + vertices[i][dim]
    return R

# Function to check rigidity
def check_rigidity(vertices, edges):
    R = build_rigidity_matrix(vertices, edges)  # Build the rigidity matrix
    rank = R.rank()  # Compute the rank of the matrix
    n = len(vertices)
    expected_rank = 3 * n - 6  # Expected rank for rigidity in 3D
    is_rigid = rank == expected_rank  # Check if the graph is rigid
    return is_rigid, rank, expected_rank

# Generate non-isomorphic graphs with 9 vertices and 21 edges
graph_generator = graphs.nauty_geng("8 18")
graphs_list = list(graph_generator)

# Initialize a list to store the rigidity results
rigidity_results = []

# Process each generated graph
for i, G in enumerate(graphs_list):  
    edges = G.edges(labels=False)  # Extract edges without labels
    
    # Check rigidity for the current graph using predefined vertices
    is_rigid, rank, expected_rank = check_rigidity(vertices, edges)
    
    # Append the result (1 for rigid, 0 for flexible) to the list
    rigidity_results.append(1 if is_rigid else 0)

# Convert the list of results to a SageMath vector for easier printing
rigidity_vector = vector(rigidity_results)

# Print the rigidity results
print("Rigidity results:", rigidity_vector)

Rigidity results: (1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 

In [15]:
# Initialize a list to store the conjecture results
conjecture_results = []

# Loop over each graph in your list of graphs
for i, G in enumerate(graphs_list):
    # Extract edges without labels (i.e., ignore vertex labels)
    edges = G.edges(labels=False)
    
    # Apply the Conjecture function to each graph with its edges
    result = Conjecture(G)
    
    # Append the result (1 for rigid, 0 for non-rigid) to the list
    conjecture_results.append(result)

# Convert the list of results to a SageMath vector for easier printing
conjecture_vector = vector(conjecture_results)

# Print the conjecture results
print("Conjecture results:", conjecture_vector)


Conjecture results: (1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1

In [16]:
rigidity_vector-conjecture_vector

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
#NO WRONG PREDICTIONS FOR ALL CIRITICAL GRAPHS OF 8 VERTICES AND 18 EDGES!